In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

In [0]:
# Carrega a tabela de clubes do bronze
df_clube = spark.table("project_data_football_bronze.clubes")

In [0]:
# Seleciona e renomeia colunas da tabela de clubes
df_base = df_clube.select(
    col("clube_id").cast("int").alias("clube_id"),
    col("nome_fantasia").alias("nome_completo"),
    col("abreviacao"),
    col("apelido"),
    col("slug"),
    col("dt_ingestao").alias("dt_ultima_atualizacao")
)

In [0]:
# Define janela para pegar o registro mais recente por clube
window_spec = Window.partitionBy("clube_id").orderBy(col("dt_ultima_atualizacao").desc())

In [0]:
# Seleciona o registro mais recente de cada clube
df_dim_clube = (
    df_base
        .withColumn("row_num", row_number().over(window_spec))
        .filter(col("row_num") == 1)
        .drop("row_num")
)

In [0]:
#salvar
df_dim_clube.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("project_data_football_silver.dim_clube")